In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[4],3b
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[6]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.000087                    1  0.000141
0.000750        2  0.000094                    2  0.000173
0.001052        3  0.000111                    3  0.000149
0.001476        4  0.000129                    4  0.000158
0.002070        5  0.000148                    5  0.000173
0.002904        6  0.000167                    6  0.000187
0.004074        7  0.000187                    7  0.000202
0.005714        8  0.000208                    8  0.000217
0.008015        9  0.000230                    9  0.000234
0.011243       10  0.000252                   10  0.000250
0.015771       11  0.000274                   11  0.000268
0.022122       12  0.000298                   12  0.000285
0.031031       13  0.000322                   13  0.000304
0.043528       14  0.000349                   14  0.000327
0.061057       15  0.000432                   15  0.000393
0.085645       16  0.000701                   16  0.000590
0.120136       17  0.001212                   17  0.000964
0.168516       18  0.001936                   18  0.001490
0.236378       19  0.002923                   19  0.002201
0.331549       20  0.004242                   20  0.003146
0.465100       21  0.005978                   21  0.004400
0.652400       22  0.008215                   22  0.006117
0.915100       23  0.010369                   23  0.007794
1.283650       24  0.010399                   24  0.007916
1.800600       25  0.008443                   25  0.006598
2.525700       26  0.006398                   26  0.005166
3.542800       27  0.004728                   27  0.003948
4.969550       28  0.003453                   28  0.002983
6.970850       29  0.002534                   29  0.002251
9.778100       30  0.001871                   30  0.001695
13.715850      31  0.001366                   31  0.001250
19.239350      32  0.000981                   32  0.000899
26.987250      33  0.000730                   33  0.000660
37.855300      34  0.000576                   34  0.000507
53.100050      35  0.000447                   35  0.000384
73.887500      36  0.000335                   36  0.000282
97.662500      37  0.000279                   37  0.000231
121.437500     38  0.000263                   38  0.000215
145.212500     39  0.000248                   39  0.000200
168.987500     40  0.000269                   40  0.000219
192.762500     41  0.000584                   41  0.000477
216.537500     42  0.002350                   42  0.002016
240.312500     43  0.006827                   43  0.006293
264.087500     44  0.015031                   44  0.014831
287.862500     45  0.026836                   45  0.027342
311.637500     46  0.035972                   46  0.036438
335.412500     47  0.044634                   47  0.044796
359.187500     48  0.054799                   48  0.055617
382.962500     49  0.066441                   49  0.068167
406.737500     50  0.079516                   50  0.080553
430.512500     51  0.093934                   51  0.091921
454.287500     52  0.107622                   52  0.101485
478.062500     53  0.121814                   53  0.112267
501.837500     54  0.137498                   54  0.126228
525.612500     55  0.154630                   55  0.143318
549.387500     56  0.173164                   56  0.162715
573.162500     57  0.193229                   57  0.183027
596.937500     58  0.223813                   58  0.211656
620.712500     59  0.255233                   59  0.238962
644.487500     60  0.285041                   60  0.263735
668.262500     61  0.313058                   61  0.288015
692.037500     62  0.338756                   62  0.313258
715.812500     63  0.362089                   63  0.340769
739.587500     64  0.383002                   64  0.369449
763.362500     65  0.401405                   65  0.396800
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -38.664460  0.000000e+00 -38.664460 -38.767894   
0.000624    2     -38.664460  5.026141e-08 -38.664460 -38.767894   
0.000876    3     -38.664460  7.073615e-08 -38.664460 -38.767894   
0.001229    4     -38.664460  1.001601e-07 -38.664460 -38.767894   
0.001723    5     -38.664460  1.425242e-07 -38.664460 -38.767894   
0.002417    6     -38.664460  2.035053e-07 -38.664460 -38.767895   
0.003391    7     -38.664461  2.912839e-07 -38.664460 -38.767895   
0.004757    8     -38.664461  4.176285e-07 -38.664460 -38.767895   
0.006672    9     -38.664461  5.994596e-07 -38.664460 -38.767895   
0.009359    10    -38.664461  8.611365e-07 -38.664460 -38.767895   
0.013128    11    -38.664461  1.237692e-06 -38.664460 -38.767895   
0.018415    12    -38.664462  1.779531e-06 -38.664460 -38.767895   
0.025830    13    -38.664462  2.559154e-06 -38.664459 -38.767895   
0.036232    14    -38.664463  3.680864e-06 -38.664459 -38.767896   
0.050823    15    -38.664464  5.297334e-06 -38.664458 -38.767896   
0.071291    16    -38.664465  7.747969e-06 -38.664457 -38.767897   
0.100000    17    -38.664467  1.197590e-05 -38.664455 -38.767898   
0.140271    18    -38.664469  1.987057e-05 -38.664449 -38.767899   
0.196760    19    -38.664471  3.458318e-05 -38.664436 -38.767899   
0.275997    20    -38.664471  6.179104e-05 -38.664409 -38.767897   
0.387100    21    -38.664465  1.116186e-04 -38.664353 -38.767890   
0.543100    22    -38.664445  2.020829e-04 -38.664243 -38.767872   
0.761700    23    -38.664394  3.639472e-04 -38.664030 -38.767830   
1.068500    24    -38.664288  6.349934e-04 -38.663653 -38.767747   
1.498800    25    -38.664130  1.007474e-03 -38.663123 -38.767628   
2.102400    26    -38.663953  1.433750e-03 -38.662519 -38.767499   
2.949000    27    -38.663772  1.894087e-03 -38.661878 -38.767372   
4.136600    28    -38.663595  2.383081e-03 -38.661213 -38.767259   
5.802500    29    -38.663432  2.902003e-03 -38.660531 -38.767169   
8.139200    30    -38.663291  3.462110e-03 -38.659829 -38.767112   
11.417000   31    -38.663183  4.080672e-03 -38.659103 -38.767103   
16.014700   32    -38.663134  4.773987e-03 -38.658359 -38.767165   
22.464000   33    -38.663175  5.565006e-03 -38.657610 -38.767329   
31.510500   34    -38.663344  6.515893e-03 -38.656828 -38.767629   
44.200100   35    -38.663685  7.724519e-03 -38.655961 -38.768114   
62.000000   36    -38.664296  9.278571e-03 -38.655017 -38.768870   
85.775000   37    -38.665271  1.119760e-02 -38.654073 -38.769974   
109.550000  38    -38.666357  1.306984e-02 -38.653287 -38.771146   
133.325000  39    -38.667521  1.497463e-02 -38.652547 -38.772362   
157.100000  40    -38.668757  1.690719e-02 -38.651850 -38.773610   
180.875000  41    -38.670100  1.900851e-02 -38.651091 -38.774941   
204.650000  42    -38.672164  2.271600e-02 -38.649447 -38.776974   
228.425000  43    -38.677404  3.457608e-02 -38.642828 -38.782254   
252.200000  44    -38.688480  6.487949e-02 -38.623601 -38.793594   
275.975000  45    -38.708999  1.277301e-01 -38.581269 -38.814767   
299.750000  46    -38.743558  2.378681e-01 -38.505690 -38.850497   
323.525000  47    -38.789629  3.852467e-01 -38.404382 -38.897966   
347.300000  48    -38.846193  5.675148e-01 -38.278678 -38.955742   
371.075000  49    -38.914115  7.897675e-01 -38.124348 -39.024675   
394.850000  50    -38.994131  1.056902e+00 -37.937229 -39.105483   
418.625000  51    -39.086813  1.373527e+00 -37.713286 -39.198579   
442.400000  52    -39.192731  1.743993e+00 -37.448739 -39.304354   
466.175000  53    -39.310149  2.164508e+00 -37.145641 -39.421141   
489.950000  54    -39.438278  2.635705e+00 -36.802574 -39.548099   
513.725000  55    -39.577258  3.161922e+00 -36.415335 -39.685436   
537.500000  56    -39.727087  3.747240e+00 -35.979847 -39.833066   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')